In [1]:
import numpy as np
import math

def Q(x):
    """Gaussian Q-function using the complementary error function."""
    return 0.5 * math.erfc(x / math.sqrt(2))

def capacity(gamma):
    """Shannon capacity C(gamma) = log2(1 + gamma)."""
    return math.log2(1 + gamma)

def var_phi(gamma):
    """Channel dispersion V(gamma) = gamma(gamma+2)/[2(gamma+1)^2] * (log2(e))^2."""
    return (gamma * (gamma + 2)) / (2 * (gamma + 1)**2) * (math.log2(math.e) ** 2)


In [2]:
def compute_sic_phi(U, eta, p, g, H=None, B=1, sigma2=1e-9, L=256, n=512):
    """
    Perform SIC decoding and return success flags phi for each user.
    
    Inputs:
    - U: list of user IDs (e.g., [1,2,...]).
    - eta: dict mapping user -> received power (float).
    - p: dict mapping user -> transmit power (float).
    - g: dict mapping user -> large-scale fading (float).
    - H: (optional) dict mapping user -> complex channel vector (np.array).
         If None, random complex vectors of dimension M=4 are generated.
    - B: maximum number of multiplexed users to consider.
    - sigma2: AWGN noise variance.
    - L: packet length in bits.
    - n: blocklength (number of channel uses).
    
    Output:
    - phi: dict mapping user -> 0 or 1 (decoding outcome).
    """
    # Initialize all phi[k] = 0
    phi = {k: 0 for k in U}
    
    # Generate random channel vectors if not provided
    if H is None:
        M = 4  # dimension of channel vectors
        H = {}
        for k in U:
            # Random complex Gaussian channel vector
            H[k] = np.random.randn(M) + 1j * np.random.randn(M)
    
    # Sort users by descending received power (only keep top B if needed)
    sorted_users = sorted(U, key=lambda k: eta[k], reverse=True)
    if len(sorted_users) > B:
        sorted_users = sorted_users[:B]
    
    # Loop over users in SIC order
    for i, k in enumerate(sorted_users):
        J1 = sorted_users[:i]      # Users decoded before user k
        J2 = sorted_users[i+1:]    # Users to be decoded after user k
        
        # Compute denominator: noise plus interference from J1 and J2
        denom = sigma2
        hk = H[k]
        
        # Add interference from J1 (only if those were not decoded successfully)
        for j in J1:
            hj = H[j]
            # eta_{jk} = p_j * g_j * |h_k^H h_j|^2 / ||h_k||^2
            eta_jk = p[j] * g[j] * (abs(np.vdot(hk, hj))**2) / (np.linalg.norm(hk)**2)
            denom += (1 - phi[j]) * eta_jk
        
        # Add interference from J2 (none of these are decoded yet)
        for j in J2:
            hj = H[j]
            eta_jk = p[j] * g[j] * (abs(np.vdot(hk, hj))**2) / (np.linalg.norm(hk)**2)
            denom += eta_jk
        
        # Compute SINR for user k
        gamma_k = eta[k] / denom
        
        # Compute error probability epsilon_k via normal approximation
        C = capacity(gamma_k)
        V = var_phi(gamma_k)
        q_arg = math.sqrt(n / V) * (C - L / n)
        epsilon = Q(q_arg)
        
        # Perform Bernoulli trial: success with probability (1 - epsilon)
        phi[k] = 1 if np.random.rand() < (1 - epsilon) else 0
    
    return phi


In [22]:
# Define inputs for a small example
U = [1, 2, 3]
eta = {1: 5.0, 2: 2.0, 3: 1.0}     # received powers
p = {1: 1.0, 2: 0.5, 3: 0.2}       # transmit powers
g = {1: 1.0, 2: 0.8, 3: 0.5}       # large-scale fading
# Optionally provide explicit channel vectors, or omit H to use random
H = {
    1: np.array([1+1j, 0.5+0.2j, 0.1+0.3j]),
    2: np.array([0.7+0.4j, -0.2+0.8j, 0.5-0.5j]),
    3: np.array([-0.3+0.1j, 0.9+0.2j, 0.1+0.1j])
}
B = 3
sigma2 = 1e-9
L = 256
n = 512

phi_out = compute_sic_phi(U, eta, p, g, H, B, sigma2, L, n)
print(phi_out)  # e.g., {1: 1, 2: 0, 3: 1}


{1: 1, 2: 1, 3: 1}
